In [17]:
import rasterio
import geopandas as gpd
import pandas as pd
import numpy as np
import folium
import matplotlib.colors as mcolors
from rasterio.plot import show
from rasterio.transform import rowcol

In [24]:
filepath = "/Users/richie/Projects/smart-green-city-konstanz/richards_notebooks/LCZ/data/messstationen_meta.csv"
mess_meta = pd.read_csv(filepath, sep=";")
mess_meta
print(mess_meta[['Geogr.Laenge', 'Geogr.Breite']].describe())

       Geogr.Laenge  Geogr.Breite
count     11.000000     11.000000
mean       9.179880     47.668770
std        0.019245      0.008217
min        9.147058     47.658239
25%        9.171379     47.662143
50%        9.177239     47.667215
75%        9.185754     47.673042
max        9.214489     47.682119


In [14]:
#Maja
import geopandas as gpd
import matplotlib.pyplot as plt

#LCZ-Raster laden (Koordinatenreferenzsystem: EPSG:4326)
lcz_path ="/Users/richie/Projects/smart-green-city-konstanz/richards_notebooks/LCZ/data/4381f9078b2704eac4c9ab8c1631439c4ccff939.tif"

#Kleinräumige Gliederung (?) 
geojson_path = "/Users/richie/Projects/smart-green-city-konstanz/majas_notebooks/Kleinräumige_Gliederung_4330766187849128049.geojson"

# Stadtteile von Konstanz laden (GeoJSON-Datei erforderlich)
stadtteile = gpd.read_file(geojson_path)

# raster laden
raster = rasterio.open(lcz_path)

named_coordinates = list(zip(mess_meta["name"],mess_meta["Geogr.Laenge"], mess_meta["Geogr.Breite"]))

# Koordinaten nutzen um Local Climate Zone Werte aus Raster zu extrahieren
results = []
for name,breite,länge in named_coordinates:
    # Koordinaten in Pixel (Zeile, Spalte) im Raster umwandeln
    row, col = rowcol(raster.transform, breite, länge)
    
    # LCZ-Wert an dieser Position auslesen
    lcz_value = raster.read(1)[row, col]
    
    # Ergebnis mit Namen speichern
    results.append({"Name": name, "Geografische Breite": breite, "Geografische Länge": länge, "LCZ_Class": lcz_value})

results

[{'Name': 'Döbele',
  'Geografische Breite': 9.16889836,
  'Geografische Länge': 47.65823856,
  'LCZ_Class': np.int16(15)},
 {'Name': 'Europapark',
  'Geografische Breite': 9.16282324,
  'Geografische Länge': 47.66721463,
  'LCZ_Class': np.int16(9)},
 {'Name': 'Fähre Staad',
  'Geografische Breite': 9.20933772,
  'Geografische Länge': 47.68211852,
  'LCZ_Class': np.int16(5)},
 {'Name': 'Friedrichstrasse',
  'Geografische Breite': 9.18375494,
  'Geografische Länge': 47.67557538,
  'LCZ_Class': np.int16(5)},
 {'Name': 'Herose-Park',
  'Geografische Breite': 9.17457428,
  'Geografische Länge': 47.66905485,
  'LCZ_Class': np.int16(9)},
 {'Name': 'Hörnle',
  'Geografische Breite': 9.21448949,
  'Geografische Länge': 47.66702444,
  'LCZ_Class': np.int16(9)},
 {'Name': 'Mainaustrasse',
  'Geografische Breite': 9.1877523,
  'Geografische Länge': 47.67050958,
  'LCZ_Class': np.int16(9)},
 {'Name': 'Marktstätte',
  'Geografische Breite': 9.17723865,
  'Geografische Länge': 47.6604731,
  'LCZ_Cla

In [ ]:

# 12 Beispielkoordinaten für Messstationen (müssen echte Werte ersetzt werden) # sind das nicht die echten Werte? 
messstationen = gpd.GeoDataFrame(
    mess_meta,
    geometry=gpd.points_from_xy(mess_meta["Geogr.Laenge"], mess_meta["Geogr.Breite"]),
    crs="EPSG:4326"
)

# Karte erstellen
fig, ax = plt.subplots(figsize=(10, 12))
stadtteile.plot(ax=ax, color="lightgrey", edgecolor="black")
# Stadtteilnamen hinzufügen
for idx, row in stadtteile.iterrows():
    centroid = row.geometry.centroid
    ax.text(centroid.x, centroid.y, row["STT_NAME"], fontsize=6, ha='center', va='center', color='black', bbox=dict(facecolor='white', alpha=0.7, edgecolor='none'))


messstationen.plot(ax=ax, color="red", marker="o", label="Messstationen")
for idx, row in messstationen.iterrows():
    ax.text(row.geometry.x, row.geometry.y, row["name"], fontsize=8, ha='right', va='bottom', color='black')

# Titel und Legende
ax.set_title("Stadtteile von Konstanz mit Messstationen")
plt.legend()
plt.show()


In [23]:
print(messstationen.geometry.type)

0     MultiPolygon
1          Polygon
2     MultiPolygon
3          Polygon
4          Polygon
5          Polygon
6          Polygon
7          Polygon
8          Polygon
9          Polygon
10    MultiPolygon
11         Polygon
12         Polygon
13         Polygon
14         Polygon
dtype: object


In [22]:
print(messstationen.geometry.geom_type.unique())

['MultiPolygon' 'Polygon']


In [21]:
#ChatGPT überlappen
import folium
import rasterio
import geopandas as gpd
import numpy as np
from rasterio.plot import show
from folium.plugins import MousePosition

# 📌 1. LCZ-Raster öffnen & speichern als PNG für Overlay

png_path = "lcz_overlay.png"

with rasterio.open(lcz_path) as src:
    lcz_data = src.read(1)
    lcz_transform = src.transform
    bounds = src.bounds

    # Farbschema für das Raster
    import matplotlib.pyplot as plt
    import matplotlib.colors as mcolors
    
    lcz_colors = {
        1: "#8B0000", 2: "#FF4500", 3: "#FFD700",
        4: "#228B22", 5: "#00FA9A", 6: "#00CED1",
        7: "#1E90FF", 8: "#4169E1", 9: "#8A2BE2",
        10: "#A52A2A", 11: "#006400", 12: "#32CD32",
        13: "#ADFF2F", 14: "#FFFF00", 15: "#D3D3D3",
        16: "#DEB887", 17: "#4682B4"
    }

    cmap = mcolors.ListedColormap([lcz_colors[i] for i in sorted(lcz_colors.keys())])
    bounds_values = np.arange(1, 18)
    norm = mcolors.BoundaryNorm(bounds_values, cmap.N)

    # Speichern des Rasters als PNG
    fig, ax = plt.subplots(figsize=(10, 10))
    show(lcz_data, transform=lcz_transform, cmap=cmap, norm=norm, ax=ax)
    plt.axis("off")
    plt.savefig(png_path, bbox_inches="tight", pad_inches=0, dpi=300)
    plt.close()

# 📌 2. Folium-Karte erstellen
m = folium.Map(location=[(bounds.top + bounds.bottom) / 2, (bounds.left + bounds.right) / 2], zoom_start=12)

# 📌 3. Raster als Bild-Overlay einfügen
image_overlay = folium.raster_layers.ImageOverlay(
    image=png_path,
    bounds=[[bounds.bottom, bounds.left], [bounds.top, bounds.right]],
    opacity=0.6,
    interactive=True
)
image_overlay.add_to(m)

# 📌 4. Stadtteile & Messstationen hinzufügen
stadtteile = gpd.read_file(geojson_path)
#messstationen = gpd.read_file(geojson_path)

# Stadtteile als Layer
folium.GeoJson(stadtteile, name="Stadtteile", style_function=lambda x: {"color": "black", "weight": 1}).add_to(m)

# Messstationen als Marker hinzufügen
for _, row in messstationen.iterrows():
    folium.Marker(
        location=[row.geometry.y, row.geometry.x],
        popup=row["name"],
        icon=folium.Icon(color="red", icon="info-sign")
    ).add_to(m)

# 📌 5. Zusätzliche Funktionen: Mausposition & Layer-Kontrolle
MousePosition().add_to(m)
folium.LayerControl().add_to(m)

# 📌 6. Karte anzeigen
m

AttributeError: 'MultiPolygon' object has no attribute 'y'